In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import scienceplots
%matplotlib inline
from seaborn import displot, histplot
plt.style.use(['seaborn-v0_8-paper'])
from tqdm import tqdm
from shapely.geometry import LineString
from conflict import ConflictLyft

with open('./lyft_avhv_conflict.pkl', 'rb') as file:
    lyft_avhv_conflict_list = pickle.load(file)

with open('./lyft_hvhv_conflict.pkl', 'rb') as file:
    lyft_hvhv_conflict_list = pickle.load(file)

lyft_conflict_list = lyft_avhv_conflict_list + lyft_hvhv_conflict_list

# convert the type to latest conflict object
conflict_list = list()
for c_index, c in enumerate(lyft_conflict_list):
    # print(c_index)
    conflict_list.append(ConflictLyft(
        leader_id=c.first_id,
        follower_id=c.second_id,
        leader_time_at_conflict=c.first_time_at_conflict,
        follower_time_at_conflict=c.second_time_at_conflict,
        leader_is_av=c.is_first_AV,
        follower_is_av=c.is_second_AV,
        PET=c.PET,
        conflict_type=c.category.upper(),
        leader_traj=c.first_trajectory.xy[1:-1,:],
        follower_traj=c.second_trajectory.xy[1:-1,:],
        leader_traj_timestamp=c.first_trajectory.t[1:-1],
        follower_traj_timestamp=c.second_trajectory.t[1:-1],
        leader_traj_speed=c.first_trajectory.smoothed_v,
        follower_traj_speed=c.second_trajectory.smoothed_v,
    ))
assert len(conflict_list) == len(lyft_conflict_list)

C:\Users\LocalAdmin\Documents\GitHub\lyft_intersection\extract_intersections\conflict.py:172: RuntimeWarning: divide by zero encountered in scalar divide
  follower_trajectory_length / self.follower_beforeConflict_speeds[follower_index] - \
C:\Users\LocalAdmin\Documents\GitHub\lyft_intersection\extract_intersections\conflict.py:173: RuntimeWarning: divide by zero encountered in scalar divide
  leader_trajectory_length / self.leader_beforeConflict_speeds[leader_index]
C:\Users\LocalAdmin\Documents\GitHub\lyft_intersection\extract_intersections\conflict.py:157: RuntimeWarning: divide by zero encountered in scalar divide
  TTCs.append(follower_trajectory_length / (self.follower_beforeConflict_speeds[follower_index]))


In [ ]:
"""
Construct a dataframe with the following columns:
- Conflict type
- Vehicle order
- PET
- TTC
"""
conflict_cols_in_df = [
    "Conflict type",
    "Vehicle order",
    "PET",
    "TTC"
]

# generate each rows in the dataframe
conflict_rows_in_df = list()
for conflict in conflict_list:
    timeToCollision_list = conflict.TTCs
    leader_type = "AV" if conflict.leader_is_av else "HV"
    follower_type = "AV" if conflict.follower_is_av else "HV" 
    conflict_type = "Cross" if conflict.conflict_type == "CROSS" else "Merge"
    PET = conflict.PET
    for TTC in timeToCollision_list:
        conflict_rows_in_df.append([
                conflict_type,
                leader_type+"-"+follower_type,
                PET,
                TTC,
            ])

conflict_df = pd.DataFrame(conflict_rows_in_df, columns=conflict_cols_in_df)

# Post-process
PET_THRESHOLD = 10
conflict_df = conflict_df[conflict_df["PET"] <= PET_THRESHOLD]

# display info
conflict_df.info(), conflict_df.shape

In [ ]:
# Post-process: remove TTC greater than 10s
TTC_THRESHOLD = 10
conflict_df = conflict_df[conflict_df["TTC"] <= TTC_THRESHOLD]

# display info
conflict_df.info()

In [ ]:
conflict_df["Vehicle order"].value_counts()

In [ ]:
palette = {
    "HV-HV" : '#257ab6', 
    "AV-HV" : '#31a231',
    "HV-AV" : '#ff8214',
}

HVHV_COLOR = '#257ab6'
HVAV_COLOR = '#31a231'
AVHV_COLOR = '#ff8214'

FIGURE_WIDTH, FIGURE_HEIGHT = 3, 3
conflictType = "Total"
plt.figure(figsize=(FIGURE_WIDTH, FIGURE_HEIGHT))

displot(
    data=conflict_df,
    x="TTC", 
    hue="Vehicle order",
    hue_order=["HV-HV", "AV-HV", "HV-AV"],
    kind="kde",
    common_norm=False,
    palette=palette,
    height=FIGURE_HEIGHT,
    aspect=1.0,
    legend=False,
)

plt.xlim([0, TTC_THRESHOLD])
# plt.ylim([0,0.15])
plt.legend(["HV-AV", "AV-HV", "HV-HV"], title="Vehicle order")
plt.title(f"TTC distribution [{conflictType}] (Lyft)")
# plt.savefig(f"./outputs/plots/lyft_distribution_TTC_{conflictType}.png", dpi=300, bbox_inches="tight")
plt.show()